___
<a href='https://agroinformatics.org'> <img src='images/GEMS-logo2d.png' /></a>
___

# spaCy Basics

**spaCy** (https://spacy.io/) is an open-source Python library that parses and "understands" large volumes of text. 


# Installation and Setup

Installation is a two-step process. First, install spaCy using either conda or pip. Next, download the specific model you want, based on language.<br> For more info visit https://spacy.io/usage/

### 1. From the command line or terminal:
> `conda install -c conda-forge spacy`
> <br>*or*<br>
> `pip install -U spacy`

> ### Alternatively you can create a virtual environment:
> `conda create -n spacyenv python=3 spacy=2`

### 2. Next, also from the command line (you must run this as admin or use sudo):

> `python -m spacy download en`

> ### If successful, you should see a message like:

> **`Linking successful`**<br>
> `    PathToSpacyEnvironment\spacyenv\lib\site-packages\en_core_web_sm -->`<br>
> `    PathToSpacyEnvironment\spacyenv\lib\site-packages\spacy\data\en`<br>
> ` `<br>
> `    You can now load the model via spacy.load('en')`

### 3. To check version of spacy on the command line:
`python3 -m spacy info`


# Working with spaCy in Python

This is a typical set of instructions for importing and working with spaCy. Will likely take awhile - spaCy has a fairly large library to load:

In [ ]:
import PyPDF2
import random
import re

# Import spaCy
import spacy
# Import visualization package
from spacy import displacy


# Load the language library
nlp = spacy.load('en_core_web_sm')

# Create a Doc object
doc = nlp(u'G.E.M.S is applying for an NSF grant worth $5 million. Good luck!')

# Check to see what components are current live
nlp.pipe_names

In [ ]:
# Print each token separately
for token in doc:
    print(token.text, token.pos_, token.dep_)

You can use the explain function in spacy to find out meaning of tags

In [ ]:
spacy.explain('PROPN')

Right away we see some interesting things happen:
1. G.E.M.S is recognized to be a Proper Noun, not just a word at the start of a sentence
2. G.E.M.S is kept together as one entity ('token'). Not broken on periods. 

___
# spaCy Objects

After importing the spacy module in the cell above we loaded a **model** and named it `nlp`.<br>Next we created a **Doc** object by applying the model to our text, and named it `doc`.<br>

___
# Pipeline
When we run `nlp`, our text enters a *processing pipeline* that first breaks down the text and then performs a series of operations to tag, parse and describe the data.   Image source: https://spacy.io/usage/spacy-101#pipelines

<img src="images/pipeline1.png" width="600">

We can check to see what components currently live in the pipeline. 

In [ ]:
nlp.pipeline

___
## Tokenization
The first step in processing text is to split up all the component parts (words & punctuation) into "tokens". These tokens are annotated inside the Doc object to contain descriptive information. 

In [ ]:
# NOTE: I have no idea if this is true. Kevin and Jesse would know better. Just demonstrating Spacy. 
doc2 = nlp(u"G.E.M.S isn't looking into hiring anymore.")

for token in doc2:
    print(token.text, token.pos_, token.dep_)

Notice how `isn't` has been split into two tokens. spaCy recognizes both the root verb `is` and the negation attached to it. Notice also that both the extended whitespace and the period at the end of the sentence are assigned their own tokens. Spacy is able to recognize the extended space is different than space between words. For texts such as poems this might be useful.v

NOTE: Even though `doc2` contains processed information about each token, it also retains the original text:

In [ ]:
doc2

In [ ]:
doc2[0]

In [ ]:
type(doc2)

___
## Part-of-Speech Tagging (POS)
The next step after splitting the text up into tokens is to assign parts of speech. In the above example, `G.E.M.S` was recognized to be a ***proper noun***. Here some statistical modeling is required. For example, words that follow "the" are typically nouns. A big advantage of using Spacy is we did not have to do any training here. 

For a full list of POS Tags visit https://spacy.io/api/annotation#pos-tagging

In [ ]:
doc2[0].pos_

___
## Dependencies
In addition to tagging, Spacy assigns syntactic dependencies to each token. This is the relationship between words. Notice below Spacy is able to recognize the word/token Great has two different relationships in the two sentences. 

For a full list of Syntactic Dependencies visit https://spacy.io/api/annotation#dependency-parsing
<br>A good explanation of typed dependencies can be found [here](https://nlp.stanford.edu/software/dependencies_manual.pdf)

In [ ]:
doc2 = nlp(u"Great Engineers are in demand.")
doc3 = nlp(u"Great Britain is in trouble.")

In [ ]:
print(doc2[0].text, ",", doc2[0].pos_, ",", doc2[0].dep_)

In [ ]:
print(doc3[0].text, ",", doc3[0].pos_, ",", doc3[0].dep_)

To see the full name of a tag use `spacy.explain(tag)`

In [ ]:
print("ADJ =",spacy.explain('ADJ'))
print("amod =",spacy.explain('amod'))
print("PROPN =",spacy.explain('PROPN'))
print("compound =",spacy.explain('compound'))

### Visualizing dependencies

In [ ]:
# Render the dependency parse immediately inside Jupyter:
displacy.render(doc2, style='dep', jupyter=True, options={'distance': 150})

___
## Additional Token Attributes
Some of the other information that spaCy assigns to tokens:

In [ ]:
# Lemmas (the base form of the word):
print(doc2[2].text)
print(doc2[2].lemma_)

In [ ]:
# Simple Parts-of-Speech & Detailed Tags:
print(doc2[2].pos_)
print(doc2[2].tag_ + ' / ' + spacy.explain(doc2[2].tag_))

In [ ]:
# Boolean Values:
print(doc2[0].is_alpha)
print(doc2[0].is_stop)
print(doc2[3].is_stop)

## Named Entity Recognition (NER)
spaCy has an **'ner'** pipeline component that identifies token spans fitting a predetermined set of named entities. These are available as the `ents` property of a `Doc` object.

In [ ]:
# Import spaCy and load the language library
import spacy
nlp = spacy.load('en_core_web_sm')

# Create a Doc object
doc = nlp(u'G.E.M.S is applying for an NSF grant worth $5 million.')

for ent in doc.ents:
    print(ent.text+' - '+ent.label_+' - '+str(spacy.explain(ent.label_)))

### Visualizing (NER)

In [ ]:
displacy.render(doc, style='ent', jupyter=True)

___
## Spans
Large Doc objects can be hard to work with at times. A **span** is a slice of Doc object in the form `Doc[start:stop]`.

In [ ]:
doc3 = nlp(u'One of the reasons I enjoy working with this group is its diverse pool of talent.  \
To quote Karl Popper “Disciplines are distinguished partly for historical reasons and reasons of \
administrative convenience.......We are not students of some subject matter but students of problems. \
And problems may cut right across the borders of any subject matter or discipline” ')

In [ ]:
quote_source = doc3[20:22]
print(quote_source)

In [ ]:
type(quote_source)

## Aside: Generators
For efficiency, Spacy uses a lot of generators. This reduces memory foot print. Data are not generated unti when they are needed. <br/>

### Example: Given a list x = [1, 2, 3] <br/>
We know a user might need to use the square of the first two items. 

### Solution 1: Write a function


In [ ]:
def doubleListFunc(lst):
    newList = []
    for item in lst:
        newList.append(item * 2)
    return newList

x = [4, 12, 3]
x1 = doubleListFunc(x) # This solution doubles everything in the list even if we might only need 1st item
print(x1[0])

### Solution 2: Write a generator

In [ ]:
def doubleListGen(lst):
    for item in lst:
        yield item * 2
        
x = [4, 12, 3]
our_generator = doubleListGen(x)
next(our_generator) # Gets excuted when the item is needed. 

In [ ]:
next(our_generator)

In [ ]:
next(our_generator)

In [ ]:
# next(our_generator) # This will give an error. Generator got to the end of the list

### With a generator, if you need all the items, you will need a loop

In [ ]:
x = [4, 12, 3]
x2 = []
our_generator = doubleListGen(x)
for x in our_generator:
    x2.append(x)
print(x2[2])

### Or, use list comprehension

In [ ]:
x = [4, 12, 3]
our_generator = doubleListGen(x)
x2 = [x for x in our_generator]
print(x2[2])

___
## Sentences
Certain tokens inside a Doc object may also receive a "start of sentence" tag. While this doesn't immediately build a list of sentences, these tags enable the generation of sentence segments through `Doc.sents`. 

In [ ]:
doc4 = nlp(u'This is the first sentence. This is another sentence. This is the last sentence.')

In [ ]:
#doc4.sents[0]
for sent in doc4.sents:
     print(sent)

In [ ]:
doc4[6].is_sent_start

# Extracting Barley Related Ag Data

Instructions for extracting agricultural related data from descriptions of barley cultivars. Before extracting ag data, we need to add new named entities to the training model. These entities are: <br/>
ALAS = varietal_alias <br/>
CROP = crop <br/>
CVAR = crop_variety <br/>
JRNL = journal_reference <br/>
PATH = pathogen <br/>
PED  = pedigree <br/>
PLAN = plant_anatomy <br/>
PPTD = plant_predisposition_to_disease <br/>
TRAT = trait

## Import training data and utility functions

In [ ]:
from src.trainNER import *

## Train model to recognized ag data entities e.g., TRAT

In [ ]:
# Depending on the nature of out training dataset, we might get warnings that the 
# data is not well formatted. Ignore those errors for now. 
import warnings
warnings.filterwarnings('ignore')

# NOTE: You need to change the value of output_dir to a directory in your system
output_dir="temp"
# If you have high quality data, you can increase the number of iterations to 100. We 
# will use 10 for testing just to illustrate how the training works. 
n_iter = 10
#trainModel(None,output_dir,n_iter)

## Load model 

In [ ]:
agdata_nlp = spacy.load(output_dir)
# Add a custom function to the pipeline that fixes journal and PED entities. 
# agdata_nlp.add_pipe(compound_trait_entities, after='ner')
# Confirm custom function was added to the pipeline. 
print(agdata_nlp.pipe_names)

## Do a simple test

In [ ]:
# Test case
test_text = '''Kold is a six-rowed winter feed barley and it was published in Crop Science 25:1123 (1985). 
It matures early and its awns are rough. It was selected from the 
cross Robust/6/Glenn/4/Nordic//Dickson/Trophy/3/Azure/5/Glenn/Karl.'''

# Run spacy pipeline on test data
doc = agdata_nlp(test_text)

# Specify colors for the different entities to use with displacy
colors = {'ALAS':'BlueViolet','CROP': 'Aqua','CVAR':'Chartreuse','PATH':'red','PED':'orange','PLAN':'pink','PPTD':'brown','TRAT':'yellow'}
cust_options = {'ents': ['ALAS','CROP','CVAR','PATH','PED','PLAN','PPTD','TRAT'], 'colors':colors}

# Visualize the entities using displacy. 
displacy.render(doc, style='ent', jupyter=True,options=cust_options)

## Try on a PDF file
### Open PDF file, extract one page, classify and display  

In [ ]:
#Open PDF file for reading
pdfFile = open("Data/DavisLJ11/BarCvDescLJ11.pdf", mode="rb")
pdfReader = PyPDF2.PdfFileReader(pdfFile)

# Find the total number 
numPages = pdfReader.numPages

# Randomly pick one if you want to. 
# pageNumber = random.randrange(0,numPages)

# For now I will pick a specific page. 
pageNumber= 24


# Get text
OnePage = pdfReader.getPage(pageNumber)
OnePageText = OnePage.extractText()

# Close PDF file
pdfFile.close()

# Remove newlines. It appears multiple newlines together makes
# Spacy think that is the end of a sentence. The PDF reader reads the text in
# an odd fashion
OnePageText = OnePageText.replace('\n','')


# Run spacy pipeline on test data
doc = agdata_nlp(OnePageText)

# Display entities if any were found. 
if doc.ents:
        displacy.render(doc, style='ent', jupyter=True, options=cust_options)
    